This data comes from ratings of bobatea shops. The data has already been cleaned (duplicates dropped, rows with empty information that would affect the results dropped which was missing name or address since rating and lon-lat are necessary, any impossible results such as a rating over 5, etc.) 

I wanted to try writing an analysis that included geospatial data. 

The project is based on a marketing analysis by the boba-guys-san-francisco shop. It is designed to find all of the boba shops within 5 miles of their location, list them in descending order of their yelp review, and then proivde full information so that the most relevant shops could be studies to see how they compete and what could be done to counter the competition. 

In [86]:
import pandas as pd
import matplotlib.pyplot as plt
from geopy.distance import geodesic
import seaborn as sns


In [76]:
boba_data = pd.read_csv("boba_shops_clean.csv")

In [77]:
print(boba_data.head())

    id                             name  rating                  address  \
0  0.0           99-tea-house-fremont-2     4.5        3623 Thornton Ave   
1  1.0                one-tea-fremont-2     4.5  46809 Warm Springs Blvd   
2  2.0             royaltea-usa-fremont     4.0       38509 Fremont Blvd   
3  3.0  teco-tea-and-coffee-bar-fremont     4.5   39030 Paseo Padre Pkwy   
4  4.0                  t-lab-fremont-3     4.0       34133 Fremont Blvd   

            city   latitude   longitude  corrected longitude  
0        Fremont  37.562950  122.010040          -122.010040  
1        Fremont  37.489067  121.929414          -121.929414  
2        Fremont  37.551315  121.993850          -121.993850  
3        Fremont  37.553694  121.981043          -121.981043  
4        Fremont  37.576149  122.043705          -122.043705  


In [78]:
lat_data = boba_data.latitude
print(lat_data)
lat_data_clean = lat_data[:595]
print(lat_data_clean)
print(len(lat_data_clean))

0      37.562950
1      37.489067
2      37.551315
3      37.553694
4      37.576149
         ...    
593    37.673550
594    37.700116
595          NaN
596          NaN
597          NaN
Name: latitude, Length: 598, dtype: float64
0      37.562950
1      37.489067
2      37.551315
3      37.553694
4      37.576149
         ...    
590    37.631869
591    37.679500
592    37.651128
593    37.673550
594    37.700116
Name: latitude, Length: 595, dtype: float64
595


In [79]:
lon_data = boba_data["corrected longitude"]
print(lon_data)
lon_data_clean = lon_data[:595]
print(lat_data_clean)
print(len(lon_data_clean))

0     -122.010040
1     -121.929414
2     -121.993850
3     -121.981043
4     -122.043705
          ...    
593   -122.081140
594   -122.126858
595           NaN
596           NaN
597           NaN
Name: corrected longitude, Length: 598, dtype: float64
0      37.562950
1      37.489067
2      37.551315
3      37.553694
4      37.576149
         ...    
590    37.631869
591    37.679500
592    37.651128
593    37.673550
594    37.700116
Name: latitude, Length: 595, dtype: float64
595


In [80]:
shops = boba_data.name
print(shops)
shops_clean = shops[:595]
print(shops_clean)
print(len(shops_clean))

0                  99-tea-house-fremont-2
1                       one-tea-fremont-2
2                    royaltea-usa-fremont
3         teco-tea-and-coffee-bar-fremont
4                         t-lab-fremont-3
                      ...                
593                 yogurt-hill-hayward-4
594    alohana-hawaiian-grill-san-leandro
595                                   NaN
596                                   NaN
597        Ratings over 5 (max allowable)
Name: name, Length: 598, dtype: object
0                  99-tea-house-fremont-2
1                       one-tea-fremont-2
2                    royaltea-usa-fremont
3         teco-tea-and-coffee-bar-fremont
4                         t-lab-fremont-3
                      ...                
590                         munch-hayward
591     foodnet-supermarket-san-leandro-2
592                       yo-bowl-hayward
593                 yogurt-hill-hayward-4
594    alohana-hawaiian-grill-san-leandro
Name: name, Length: 595, dtype: objec

Now that I have the data cleaned and lists I will write a function that will iterate through the list of boba tea shops and identify any that are within 5 miles of the base shop. For the purposes of this exercise, that will be shop #50, boba-guys-san-franscisco 
To calculate distance from lat-lon, I could use the Halversine formula:	
 a = sin²(Δφ/2) + cos φ1 ⋅ cos φ2 ⋅ sin²(Δλ/2);
 c = 2 ⋅ atan2( √a, √(1−a) )
 distance = R ⋅ c
where	φ is latitude, λ is longitude, R is earth’s radius (mean radius = 6,371km);
note that angles need to be in radians to pass to trig functions!
JavaScript:	
const R = 6371e3; // metres
const φ1 = lat1 * Math.PI/180; // φ, λ in radians
const φ2 = lat2 * Math.PI/180;
const Δφ = (lat2-lat1) * Math.PI/180;
const Δλ = (lon2-lon1) * Math.PI/180;

const a = Math.sin(Δφ/2) * Math.sin(Δφ/2) +
          Math.cos(φ1) * Math.cos(φ2) *
          Math.sin(Δλ/2) * Math.sin(Δλ/2);
const c = 2 * Math.atan2(Math.sqrt(a), Math.sqrt(1-a));
const d = R * c; // in metres
and then convert d to miles...

Or I could use the python geopy module which I imported in the first block of code.

In [81]:
#These lines are me testing out different parts of the code to make sure everything does what I want it to.
new_lat = lat_data_clean[0]
new_lon = lon_data_clean[0]
coordinate_start = (start_lat, start_lon)
coordiante_end = (new_lat, new_lon)
distance = geodesic(coordinate_start, coordiante_end).miles
print(distance)
print(coordiante_end)

print(len(shops_clean))
print(range(len(shops_clean)))

test = []
for i in range(20):
    test.append(i)
    
print(test)

26.819427363761253
(37.56295, -122.01003999999999)
595
range(0, 595)
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]


In [82]:
#This will now create a set of coordiantes by iterating over the list of coordinates and using geopy's geodesic function
#to determine distance between the origin shop and each shop with a sufficiently high rating.
shops_to_visit = []
for i in range(len(shops_clean)):
    start_lat = 37.7899434
    start_lon = -122.4073063
    new_lat = lat_data_clean[i]
    new_lon = lon_data_clean[i]
    coordinate_start = (start_lat, start_lon)
    coordiante_end = (new_lat, new_lon)
    distance = geodesic(coordinate_start, coordiante_end).miles
    if distance < 5:
        shops_to_visit.append(shops_clean[i])

    



In [83]:
print(shops_to_visit)

['boba-guys-san-francisco-6', 'boba-guys-san-francisco-4', 'wonderful-dessert-and-cafe-san-francisco-2', 'little-sweet-san-francisco-10', 'teaspoon-san-francisco', 'boba-guys-san-francisco-10', 'i-tea-san-francisco-3', 'boba-guys-san-francisco-7', 'plentea-san-francisco-san-francisco', 'steap-tea-bar-san-francisco-3', 'omg-tea-san-francisco', 'asha-tea-house-san-francisco', 'boba-guys-san-francisco-15', 'steep-san-francisco-2', 'mr-t-cafe-san-francisco', 'tpumps-san-francisco', 'wondertea-san-francisco-4', 'the-boba-shop-san-francisco-4', 'steep-creamery-and-tea-san-francisco', 'bubblecup-san-francisco-2', 'sweet-a-little-san-francisco-12', 'tea-hut-san-francisco-2', 'i-tea-san-francisco-2', 'tancca-san-francisco-3', 'sharetea-san-francisco-3', 'mi-tea-san-francisco', 'bb-tea-station-san-francisco-3', 'keep-it-san-francisco-6', 'mr-and-mrs-tea-house-san-francisco', 'e-tea-san-francisco', 'gosu-san-francisco', 'homeplate-boba-san-francisco', 'cuppa-san-francisco', 'puppy-bobar-san-franc

In [84]:
full_info_for_visits = boba_data[boba_data["name"].isin(shops_to_visit)]
print(full_info_for_visits)

      id                                        name  rating  \
49  50.0                   boba-guys-san-francisco-6     4.0   
50  51.0                   boba-guys-san-francisco-4     4.0   
51  52.0  wonderful-dessert-and-cafe-san-francisco-2     4.0   
52  53.0               little-sweet-san-francisco-10     4.0   
53  54.0                      teaspoon-san-francisco     4.0   
54  55.0                  boba-guys-san-francisco-10     4.0   
55  56.0                       i-tea-san-francisco-3     4.5   
56  57.0                   boba-guys-san-francisco-7     4.0   
57  58.0         plentea-san-francisco-san-francisco     3.5   
58  59.0               steap-tea-bar-san-francisco-3     4.5   
60  61.0                       omg-tea-san-francisco     4.5   
61  62.0                asha-tea-house-san-francisco     4.0   
63  64.0                  boba-guys-san-francisco-15     4.0   
64  66.0                       steep-san-francisco-2     4.0   
65  67.0                     mr-t-cafe-s

OK, now I want to get a sense for how many shops fall into each category (rating) so I group the new 
data frame by their rating number. Because the ratings are discrete (3.5, 4, 4.5, 5), they should group 
easily. I will then apply the count function to tell me how many fall into each group.

In [97]:
print(full_info_for_visits.groupby("rating").count())

        id  name  address  city  latitude  longitude  corrected longitude
rating                                                                   
3.5     15    15       15    15        15         15                   15
4.0     15    15       15    15        15         15                   15
4.5     11    11       11    11        11         11                   11
5.0      1     1        1     1         1          1                    1


Great! That's going to let me focus in on the top 12 shops which have a 4.5 or 5 star rating. To help identify these, let's sort the 
dataframe in descending order by rating.

In [85]:
full_info_for_visits = full_info_for_visits.sort_values("rating", ascending=False)
print(full_info_for_visits)

      id                                        name  rating  \
86  89.0                   puppy-bobar-san-francisco     5.0   
73  75.0                     tea-hut-san-francisco-2     4.5   
68  70.0                   wondertea-san-francisco-4     4.5   
90  93.0             good-earth-cafe-san-francisco-2     4.5   
88  91.0            little-heaven-deli-san-francisco     4.5   
55  56.0                       i-tea-san-francisco-3     4.5   
80  82.0                     keep-it-san-francisco-6     4.5   
58  59.0               steap-tea-bar-san-francisco-3     4.5   
60  61.0                       omg-tea-san-francisco     4.5   
75  77.0                      tancca-san-francisco-3     4.5   
72  74.0             sweet-a-little-san-francisco-12     4.5   
69  71.0               the-boba-shop-san-francisco-4     4.5   
94  97.0    fifty-fifty-coffee-and-tea-san-francisco     4.0   
84  86.0                homeplate-boba-san-francisco     4.0   
50  51.0                   boba-guys-san